In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
#!pip install torch torchvision opencv-python

In [3]:
import os
from PIL import Image
import torch
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split

In [4]:
class FaceDataset(Dataset):
  def __init__(self,img_dir,label_dir,img_size=448,transform=None):
    self.img_dir = img_dir
    self.label_dir = label_dir
    self.img_size=img_size
    self.transform = transform
    self.img_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]

  def __len__(self):
    return len(self.img_files)

  def __getitem__(self,idx):
    img_path = os.path.join(self.img_dir,self.img_files[idx])
    img = Image.open(img_path).convert("RGB")
    img = img.resize((self.img_size,self.img_size))

    label_path = os.path.join(self.label_dir,self.img_files[idx].replace('.jpg', '.txt'))
    boxes = []

    if os.path.exists(label_path):
      with open(label_path,'r') as file:
        for line in file:
          class_id, x_center, y_center, width, height = map(float, line.split())
          x_center *= self.img_size
          y_center *= self.img_size
          width *= self.img_size
          height *= self.img_size

          x_min = x_center - width / 2
          y_min = y_center - height / 2
          x_max = x_center + width / 2
          y_max = y_center + height / 2
          boxes.append([x_min, y_min, x_max, y_max, int(class_id)])

    boxes = np.array(boxes)

    if self.transform:
      img = self.transform(img)

    return img, boxes

In [5]:

img_dir_train=None
label_dir_train=None
img_dir_test=None
label_dir_test=None

data_transforms = transforms.Compose([
    transforms.ToTensor()
])

train_val_dataset = FaceDataset(
    img_dir=img_dir_train,
    label_dir=label_dir_train,
    transform=data_transforms
)

test_dataset = FaceDataset(
    img_dir=img_dir_test,
    label_dir=label_dir_test,
    transform=data_transforms
)

train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size

train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])